### Chapter 21: Verification Examples

This chapter provides comprehensive verification examples using SystemVerilog, demonstrating industry-standard verification methodologies and best practices.

#### Testbench for ALU

Let's start with a complete testbench for an Arithmetic Logic Unit (ALU) that demonstrates stimulus generation, checking, and coverage collection.

##### ALU Design Under Test (DUT)

```systemverilog
module alu #(
    parameter WIDTH = 8
)(
    input  logic [WIDTH-1:0] a,
    input  logic [WIDTH-1:0] b,
    input  logic [3:0]       op,
    output logic [WIDTH-1:0] result,
    output logic             zero,
    output logic             overflow,
    output logic             carry
);

    always_comb begin
        {carry, result} = '0;
        overflow = 1'b0;
        
        case (op)
            4'b0000: {carry, result} = a + b;           // ADD
            4'b0001: {carry, result} = a - b;           // SUB
            4'b0010: result = a & b;                    // AND
            4'b0011: result = a | b;                    // OR
            4'b0100: result = a ^ b;                    // XOR
            4'b0101: result = ~a;                       // NOT
            4'b0110: {carry, result} = {1'b0, a} << 1; // SHL
            4'b0111: result = a >> 1;                   // SHR
            4'b1000: result = (a < b) ? 1 : 0;         // SLT
            default: result = '0;
        endcase
        
        zero = (result == 0);
        
        // Overflow detection for addition/subtraction
        if (op == 4'b0000) // ADD
            overflow = (a[WIDTH-1] == b[WIDTH-1]) && (result[WIDTH-1] != a[WIDTH-1]);
        else if (op == 4'b0001) // SUB
            overflow = (a[WIDTH-1] != b[WIDTH-1]) && (result[WIDTH-1] != a[WIDTH-1]);
    end
endmodule
```

##### Comprehensive ALU Testbench

```systemverilog
class alu_transaction;
    rand logic [7:0] a;
    rand logic [7:0] b;
    rand logic [3:0] op;
    
    logic [7:0] result;
    logic       zero;
    logic       overflow;
    logic       carry;
    
    // Constraints
    constraint op_dist {
        op dist {
            4'b0000 := 20, // ADD
            4'b0001 := 20, // SUB
            4'b0010 := 10, // AND
            4'b0011 := 10, // OR
            4'b0100 := 10, // XOR
            4'b0101 := 10, // NOT
            4'b0110 := 10, // SHL
            4'b0111 := 10, // SHR
            4'b1000 := 10  // SLT
        };
    }
    
    constraint operand_dist {
        a dist {
            8'h00       := 5,
            8'hFF       := 5,
            [8'h01:8'hFE] := 90
        };
        b dist {
            8'h00       := 5,
            8'hFF       := 5,
            [8'h01:8'hFE] := 90
        };
    }
    
    // Display function
    function void display(string prefix = "");
        $display("%s a=%02h, b=%02h, op=%b, result=%02h, z=%b, ov=%b, c=%b",
                 prefix, a, b, op, result, zero, overflow, carry);
    endfunction
endclass

class alu_driver;
    virtual alu_if vif;
    mailbox #(alu_transaction) gen2drv;
    
    function new(virtual alu_if vif, mailbox #(alu_transaction) gen2drv);
        this.vif = vif;
        this.gen2drv = gen2drv;
    endfunction
    
    task run();
        alu_transaction trans;
        forever begin
            gen2drv.get(trans);
            drive_transaction(trans);
            #1ns; // Small delay for signal propagation
        end
    endtask
    
    task drive_transaction(alu_transaction trans);
        vif.a <= trans.a;
        vif.b <= trans.b;
        vif.op <= trans.op;
        @(posedge vif.clk);
    endtask
endclass

class alu_monitor;
    virtual alu_if vif;
    mailbox #(alu_transaction) mon2chk;
    
    function new(virtual alu_if vif, mailbox #(alu_transaction) mon2chk);
        this.vif = vif;
        this.mon2chk = mon2chk;
    endfunction
    
    task run();
        alu_transaction trans;
        forever begin
            @(posedge vif.clk);
            trans = new();
            trans.a = vif.a;
            trans.b = vif.b;
            trans.op = vif.op;
            trans.result = vif.result;
            trans.zero = vif.zero;
            trans.overflow = vif.overflow;
            trans.carry = vif.carry;
            mon2chk.put(trans);
        end
    endtask
endclass

class alu_scoreboard;
    mailbox #(alu_transaction) mon2chk;
    int pass_count = 0;
    int fail_count = 0;
    
    function new(mailbox #(alu_transaction) mon2chk);
        this.mon2chk = mon2chk;
    endfunction
    
    task run();
        alu_transaction trans;
        forever begin
            mon2chk.get(trans);
            check_result(trans);
        end
    endtask
    
    task check_result(alu_transaction trans);
        logic [8:0] expected_result;
        logic expected_zero, expected_overflow, expected_carry;
        
        case (trans.op)
            4'b0000: begin // ADD
                {expected_carry, expected_result[7:0]} = trans.a + trans.b;
                expected_overflow = (trans.a[7] == trans.b[7]) && 
                                  (expected_result[7] != trans.a[7]);
            end
            4'b0001: begin // SUB
                {expected_carry, expected_result[7:0]} = trans.a - trans.b;
                expected_overflow = (trans.a[7] != trans.b[7]) && 
                                  (expected_result[7] != trans.a[7]);
            end
            4'b0010: begin expected_result[7:0] = trans.a & trans.b; expected_carry = 0; expected_overflow = 0; end
            4'b0011: begin expected_result[7:0] = trans.a | trans.b; expected_carry = 0; expected_overflow = 0; end
            4'b0100: begin expected_result[7:0] = trans.a ^ trans.b; expected_carry = 0; expected_overflow = 0; end
            4'b0101: begin expected_result[7:0] = ~trans.a; expected_carry = 0; expected_overflow = 0; end
            4'b0110: begin {expected_carry, expected_result[7:0]} = {1'b0, trans.a} << 1; expected_overflow = 0; end
            4'b0111: begin expected_result[7:0] = trans.a >> 1; expected_carry = 0; expected_overflow = 0; end
            4'b1000: begin expected_result[7:0] = (trans.a < trans.b) ? 1 : 0; expected_carry = 0; expected_overflow = 0; end
            default: begin expected_result[7:0] = 0; expected_carry = 0; expected_overflow = 0; end
        endcase
        
        expected_zero = (expected_result[7:0] == 0);
        
        if (trans.result === expected_result[7:0] && 
            trans.zero === expected_zero && 
            trans.overflow === expected_overflow && 
            trans.carry === expected_carry) begin
            pass_count++;
            trans.display("PASS: ");
        end else begin
            fail_count++;
            trans.display("FAIL: ");
            $display("      Expected: result=%02h, z=%b, ov=%b, c=%b", 
                     expected_result[7:0], expected_zero, expected_overflow, expected_carry);
        end
    endtask
    
    function void report();
        $display("=== ALU Testbench Results ===");
        $display("PASS: %0d", pass_count);
        $display("FAIL: %0d", fail_count);
        $display("TOTAL: %0d", pass_count + fail_count);
        if (fail_count == 0)
            $display(" ALL TESTS PASSED ");
        else
            $display(" %0d TESTS FAILED ", fail_count);
    endfunction
endclass

class alu_generator;
    mailbox #(alu_transaction) gen2drv;
    int num_trans = 1000;
    
    function new(mailbox #(alu_transaction) gen2drv);
        this.gen2drv = gen2drv;
    endfunction
    
    task run();
        alu_transaction trans;
        repeat(num_trans) begin
            trans = new();
            assert(trans.randomize()) else $fatal("Randomization failed");
            gen2drv.put(trans);
        end
    endtask
endclass

interface alu_if(input logic clk);
    logic [7:0] a;
    logic [7:0] b;
    logic [3:0] op;
    logic [7:0] result;
    logic       zero;
    logic       overflow;
    logic       carry;
endinterface

// Coverage
covergroup alu_cg @(posedge alu_if.clk);
    option.per_instance = 1;
    
    cp_a: coverpoint alu_if.a {
        bins zero = {8'h00};
        bins max = {8'hFF};
        bins low = {[8'h01:8'h7F]};
        bins high = {[8'h80:8'hFE]};
    }
    
    cp_b: coverpoint alu_if.b {
        bins zero = {8'h00};
        bins max = {8'hFF};
        bins low = {[8'h01:8'h7F]};
        bins high = {[8'h80:8'hFE]};
    }
    
    cp_op: coverpoint alu_if.op {
        bins add = {4'b0000};
        bins sub = {4'b0001};
        bins and_op = {4'b0010};
        bins or_op = {4'b0011};
        bins xor_op = {4'b0100};
        bins not_op = {4'b0101};
        bins shl = {4'b0110};
        bins shr = {4'b0111};
        bins slt = {4'b1000};
    }
    
    cp_flags: coverpoint {alu_if.zero, alu_if.overflow, alu_if.carry} {
        bins no_flags = {3'b000};
        bins zero_only = {3'b100};
        bins carry_only = {3'b001};
        bins overflow_only = {3'b010};
        bins zero_carry = {3'b101};
        bins zero_overflow = {3'b110};
        bins carry_overflow = {3'b011};
        bins all_flags = {3'b111};
    }
    
    // Cross coverage
    cross_op_flags: cross cp_op, cp_flags;
endgroup

module alu_testbench();
    logic clk = 0;
    always #5ns clk = ~clk;
    
    alu_if aif(clk);
    alu_cg cg = new();
    
    alu #(.WIDTH(8)) dut (
        .a(aif.a),
        .b(aif.b),
        .op(aif.op),
        .result(aif.result),
        .zero(aif.zero),
        .overflow(aif.overflow),
        .carry(aif.carry)
    );
    
    initial begin
        mailbox #(alu_transaction) gen2drv = new();
        mailbox #(alu_transaction) mon2chk = new();
        
        alu_generator gen = new(gen2drv);
        alu_driver drv = new(aif, gen2drv);
        alu_monitor mon = new(aif, mon2chk);
        alu_scoreboard sb = new(mon2chk);
        
        fork
            gen.run();
            drv.run();
            mon.run();
            sb.run();
        join_any
        
        #100ns; // Allow final transactions to complete
        sb.report();
        $display("Coverage: %.2f%%", cg.get_coverage());
        $finish;
    end
endmodule
```

#### Memory Controller Verification

Memory controllers require sophisticated verification to ensure correct data integrity, timing, and protocol compliance.

##### Memory Controller Interface

```systemverilog
interface mem_ctrl_if(input logic clk, rst_n);
    // CPU Interface
    logic [31:0] addr;
    logic [31:0] wdata;
    logic [31:0] rdata;
    logic        we;
    logic        re;
    logic        ready;
    logic        valid;
    
    // Memory Interface
    logic [31:0] mem_addr;
    logic [31:0] mem_wdata;
    logic [31:0] mem_rdata;
    logic        mem_we;
    logic        mem_re;
    logic        mem_ready;
    
    // Control signals
    logic        refresh_req;
    logic        refresh_ack;
    
    modport cpu (
        output addr, wdata, we, re, valid,
        input  rdata, ready
    );
    
    modport mem (
        input  mem_addr, mem_wdata, mem_we, mem_re,
        output mem_rdata, mem_ready
    );
    
    modport ctrl (
        input  addr, wdata, we, re, valid, mem_rdata, mem_ready, refresh_req,
        output rdata, ready, mem_addr, mem_wdata, mem_we, mem_re, refresh_ack
    );
endinterface

class mem_transaction;
    typedef enum {READ, WRITE, REFRESH} trans_type_e;
    
    rand trans_type_e trans_type;
    rand logic [31:0] addr;
    rand logic [31:0] data;
    rand int          delay;
    
    logic [31:0] expected_data;
    logic        error;
    time         start_time;
    time         end_time;
    
    constraint addr_align { addr[1:0] == 2'b00; } // Word aligned
    constraint delay_range { delay inside {[0:10]}; }
    constraint trans_dist {
        trans_type dist {
            READ    := 40,
            WRITE   := 40,
            REFRESH := 5
        };
    }
    
    function void display(string prefix = "");
        $display("%s [%0t] Type=%s, Addr=%08h, Data=%08h, Delay=%0d", 
                 prefix, $time, trans_type.name(), addr, data, delay);
    endfunction
endclass

class mem_model;
    logic [31:0] memory [logic [31:0]];
    
    function void write(logic [31:0] addr, logic [31:0] data);
        memory[addr] = data;
        $display("[MEM_MODEL] Write: Addr=%08h, Data=%08h", addr, data);
    endfunction
    
    function logic [31:0] read(logic [31:0] addr);
        if (memory.exists(addr)) begin
            $display("[MEM_MODEL] Read: Addr=%08h, Data=%08h", addr, memory[addr]);
            return memory[addr];
        end else begin
            $display("[MEM_MODEL] Read: Addr=%08h, Data=XXXXXXXX (uninitialized)", addr);
            return 32'hXXXXXXXX;
        end
    endfunction
    
    function void clear();
        memory.delete();
    endfunction
endclass

class mem_driver;
    virtual mem_ctrl_if.cpu vif;
    mailbox #(mem_transaction) gen2drv;
    mem_model mem_ref;
    
    function new(virtual mem_ctrl_if.cpu vif, mailbox #(mem_transaction) gen2drv, mem_model mem_ref);
        this.vif = vif;
        this.gen2drv = gen2drv;
        this.mem_ref = mem_ref;
    endfunction
    
    task run();
        mem_transaction trans;
        forever begin
            gen2drv.get(trans);
            case (trans.trans_type)
                mem_transaction::READ: drive_read(trans);
                mem_transaction::WRITE: drive_write(trans);
                mem_transaction::REFRESH: drive_refresh(trans);
            endcase
        end
    endtask
    
    task drive_write(mem_transaction trans);
        trans.start_time = $time;
        
        @(posedge vif.clk);
        vif.addr <= trans.addr;
        vif.wdata <= trans.data;
        vif.we <= 1'b1;
        vif.valid <= 1'b1;
        
        wait(vif.ready);
        @(posedge vif.clk);
        vif.we <= 1'b0;
        vif.valid <= 1'b0;
        
        // Update reference model
        mem_ref.write(trans.addr, trans.data);
        trans.end_time = $time;
        trans.display("WRITE: ");
    endtask
    
    task drive_read(mem_transaction trans);
        trans.start_time = $time;
        
        @(posedge vif.clk);
        vif.addr <= trans.addr;
        vif.re <= 1'b1;
        vif.valid <= 1'b1;
        
        wait(vif.ready);
        @(posedge vif.clk);
        trans.data = vif.rdata;
        vif.re <= 1'b0;
        vif.valid <= 1'b0;
        
        trans.expected_data = mem_ref.read(trans.addr);
        trans.end_time = $time;
        trans.display("READ:  ");
    endtask
    
    task drive_refresh(mem_transaction trans);
        trans.start_time = $time;
        // Refresh is typically handled by the controller internally
        repeat(trans.delay) @(posedge vif.clk);
        trans.end_time = $time;
        trans.display("REFRESH: ");
    endtask
endclass

class mem_scoreboard;
    mailbox #(mem_transaction) mon2chk;
    int read_pass = 0, read_fail = 0;
    int write_count = 0;
    real avg_latency = 0;
    int total_latency = 0;
    
    function new(mailbox #(mem_transaction) mon2chk);
        this.mon2chk = mon2chk;
    endfunction
    
    task run();
        mem_transaction trans;
        forever begin
            mon2chk.get(trans);
            check_transaction(trans);
        end
    endtask
    
    task check_transaction(mem_transaction trans);
        int latency = (trans.end_time - trans.start_time) / 1ns;
        total_latency += latency;
        
        case (trans.trans_type)
            mem_transaction::READ: begin
                if (trans.data === trans.expected_data) begin
                    read_pass++;
                    $display("READ PASS: Addr=%08h, Data=%08h, Latency=%0d ns", 
                             trans.addr, trans.data, latency);
                end else begin
                    read_fail++;
                    $display("READ FAIL: Addr=%08h, Got=%08h, Expected=%08h", 
                             trans.addr, trans.data, trans.expected_data);
                end
            end
            mem_transaction::WRITE: begin
                write_count++;
                $display("WRITE: Addr=%08h, Data=%08h, Latency=%0d ns", 
                         trans.addr, trans.data, latency);
            end
        endcase
    endtask
    
    function void report();
        $display("=== Memory Controller Test Results ===");
        $display("Reads  - Pass: %0d, Fail: %0d", read_pass, read_fail);
        $display("Writes - Count: %0d", write_count);
        if (read_pass + write_count > 0)
            avg_latency = real'(total_latency) / real'(read_pass + write_count);
        $display("Average Latency: %.1f ns", avg_latency);
    endfunction
endclass
```

#### Bus Protocol Checker

Protocol checkers verify that bus transactions follow the specified protocol rules.

##### AXI4-Lite Protocol Checker

```systemverilog
class axi4_lite_checker;
    virtual axi4_lite_if vif;
    
    // Protocol violation counters
    int addr_phase_violations = 0;
    int data_phase_violations = 0;
    int handshake_violations = 0;
    
    function new(virtual axi4_lite_if vif);
        this.vif = vif;
    endfunction
    
    task run();
        fork
            check_write_address_channel();
            check_write_data_channel();
            check_write_response_channel();
            check_read_address_channel();
            check_read_data_channel();
        join
    endtask
    
    // Write Address Channel Checks
    task check_write_address_channel();
        forever begin
            @(posedge vif.clk);
            
            // Check: AWVALID once asserted, must remain high until AWREADY
            if (vif.awvalid && !vif.awready) begin
                logic prev_awvalid = vif.awvalid;
                logic [31:0] prev_awaddr = vif.awaddr;
                logic [2:0] prev_awprot = vif.awprot;
                
                @(posedge vif.clk);
                if (!vif.awvalid) begin
                    $error("AXI4-Lite Violation: AWVALID deasserted before AWREADY");
                    addr_phase_violations++;
                end
                
                if (vif.awvalid && (vif.awaddr !== prev_awaddr || vif.awprot !== prev_awprot)) begin
                    $error("AXI4-Lite Violation: AW signals changed while AWVALID high");
                    addr_phase_violations++;
                end
            end
            
            // Check: Address alignment
            if (vif.awvalid && vif.awaddr[1:0] !== 2'b00) begin
                $error("AXI4-Lite Violation: Unaligned address %08h", vif.awaddr);
                addr_phase_violations++;
            end
        end
    endtask
    
    // Write Data Channel Checks
    task check_write_data_channel();
        forever begin
            @(posedge vif.clk);
            
            // Check: WVALID stability
            if (vif.wvalid && !vif.wready) begin
                logic [31:0] prev_wdata = vif.wdata;
                logic [3:0] prev_wstrb = vif.wstrb;
                
                @(posedge vif.clk);
                if (vif.wvalid && (vif.wdata !== prev_wdata || vif.wstrb !== prev_wstrb)) begin
                    $error("AXI4-Lite Violation: W signals changed while WVALID high");
                    data_phase_violations++;
                end
            end
            
            // Check: Write strobe validity
            if (vif.wvalid) begin
                for (int i = 0; i < 4; i++) begin
                    if (vif.wstrb[i] && vif.wdata[i*8+7:i*8] === 8'hXX) begin
                        $warning("Write data contains X when strobe is active for byte %0d", i);
                    end
                end
            end
        end
    endtask
    
    // Write Response Channel Checks
    task check_write_response_channel();
        forever begin
            @(posedge vif.clk);
            
            // Check: BVALID stability
            if (vif.bvalid && !vif.bready) begin
                logic [1:0] prev_bresp = vif.bresp;
                
                @(posedge vif.clk);
                if (vif.bvalid && vif.bresp !== prev_bresp) begin
                    $error("AXI4-Lite Violation: BRESP changed while BVALID high");
                    handshake_violations++;
                end
            end
        end
    endtask
    
    // Read Address Channel Checks
    task check_read_address_channel();
        forever begin
            @(posedge vif.clk);
            
            // Similar checks as write address channel
            if (vif.arvalid && !vif.arready) begin
                logic [31:0] prev_araddr = vif.araddr;
                
                @(posedge vif.clk);
                if (vif.arvalid && vif.araddr !== prev_araddr) begin
                    $error("AXI4-Lite Violation: ARADDR changed while ARVALID high");
                    addr_phase_violations++;
                end
            end
        end
    endtask
    
    // Read Data Channel Checks
    task check_read_data_channel();
        forever begin
            @(posedge vif.clk);
            
            // Check: RVALID stability
            if (vif.rvalid && !vif.rready) begin
                logic [31:0] prev_rdata = vif.rdata;
                logic [1:0] prev_rresp = vif.rresp;
                
                @(posedge vif.clk);
                if (vif.rvalid && (vif.rdata !== prev_rdata || vif.rresp !== prev_rresp)) begin
                    $error("AXI4-Lite Violation: R signals changed while RVALID high");
                    data_phase_violations++;
                end
            end
        end
    endtask
    
    function void report();
        $display("=== AXI4-Lite Protocol Checker Results ===");
        $display("Address Phase Violations: %0d", addr_phase_violations);
        $display("Data Phase Violations: %0d", data_phase_violations);
        $display("Handshake Violations: %0d", handshake_violations);
        
        int total_violations = addr_phase_violations + data_phase_violations + handshake_violations;
        if (total_violations == 0)
            $display(" NO PROTOCOL VIOLATIONS DETECTED ");
        else
            $display(" %0d TOTAL PROTOCOL VIOLATIONS ", total_violations);
    endfunction
endclass
```

#### Coverage-Driven Test Scenarios

Coverage-driven verification ensures comprehensive testing by measuring what has been tested and directing stimulus generation toward untested scenarios.

##### Functional Coverage Example

```systemverilog
class cache_coverage;
    // Cache parameters
    parameter CACHE_SIZE = 1024;
    parameter LINE_SIZE = 64;
    parameter ASSOCIATIVITY = 4;
    
    // Coverage variables
    logic [31:0] addr;
    logic        hit;
    logic        miss;
    logic [1:0]  cache_state; // 00=Invalid, 01=Shared, 10=Exclusive, 11=Modified
    logic [2:0]  operation;   // 000=Read, 001=Write, 010=Prefetch, etc.
    
    covergroup cache_cg @(posedge clk);
        option.per_instance = 1;
        option.name = "cache_coverage";
        
        // Basic operation coverage
        cp_operation: coverpoint operation {
            bins read = {3'b000};
            bins write = {3'b001};
            bins prefetch = {3'b010};
            bins flush = {3'b011};
            bins invalidate = {3'b100};
        }
        
        // Cache state coverage
        cp_state: coverpoint cache_state {
            bins invalid = {2'b00};
            bins shared = {2'b01};
            bins exclusive = {2'b10};
            bins modified = {2'b11};
        }
        
        // Hit/Miss coverage
        cp_hit_miss: coverpoint {hit, miss} {
            bins hit_only = {2'b10};
            bins miss_only = {2'b01};
            illegal_bins both = {2'b11};
            ignore_bins neither = {2'b00};
        }
        
        // Address coverage - focus on cache line boundaries
        cp_address: coverpoint addr[11:6] { // Cache line index
            bins low_lines[] = {[0:15]};
            bins mid_lines[] = {[16:47]};
            bins high_lines[] = {[48:63]};
        }
        
        // Cache set coverage
        cp_cache_set: coverpoint addr[7:6] {
            bins set[] = {[0:3]};
        }
        
        // Cross coverage for state transitions
        cross_state_op: cross cp_state, cp_operation {
            bins read_hits = binsof(cp_operation) intersect {3'b000} && 
                           binsof(cp_state) intersect {2'b01, 2'b10, 2'b11};
                           
            bins write_hits = binsof(cp_operation) intersect {3'b001} && 
                            binsof(cp_state) intersect {2'b10, 2'b11};
                            
            bins cold_misses = binsof(cp_operation) intersect {3'b000, 3'b001} && 
                             binsof(cp_state) intersect {2'b00};
                             
            ignore_bins invalid_write = binsof(cp_operation) intersect {3'b001} && 
                                       binsof(cp_state) intersect {2'b00};
        }
        
        // Transition coverage
        cp_state_transitions: coverpoint cache_state {
            bins invalid_to_shared = (2'b00 => 2'b01);
            bins invalid_to_exclusive = (2'b00 => 2'b10);
            bins shared_to_modified = (2'b01 => 2'b11);
            bins exclusive_to_modified = (2'b10 => 2'b11);
            bins modified_to_invalid = (2'b11 => 2'b00);
            bins shared_to_invalid = (2'b01 => 2'b00);
            bins exclusive_to_invalid = (2'b10 => 2'b00);
        }
        
        // Address pattern coverage
        cp_addr_patterns: coverpoint addr {
            bins sequential[] = (addr[31:2] => addr[31:2] + 1);
            bins stride_2[] = (addr[31:2] => addr[31:2] + 2);
            bins stride_4[] = (addr[31:2] => addr[31:2] + 4);
            bins random_access = default;
        }
        
        // Bandwidth utilization coverage
        cp_bandwidth: coverpoint get_bandwidth_utilization() {
            bins low = {[0:25]};
            bins medium = {[26:75]};
            bins high = {[76:100]};
        }
    endgroup
    
    function int get_bandwidth_utilization();
        // Implementation specific - measure actual bandwidth vs theoretical max
        return $urandom_range(0, 100);
    endfunction
    
    cache_cg cg;
    
    function new();
        cg = new();
    endfunction
    
    function void sample(logic [31:0] addr_in, logic hit_in, logic miss_in, 
                        logic [1:0] state_in, logic [2:0] op_in);
        addr = addr_in;
        hit = hit_in;
        miss = miss_in;
        cache_state = state_in;
        operation = op_in;
        cg.sample();
    endfunction
    
    function real get_coverage();
        return cg.get_coverage();
    endfunction
    
    function void report();
        $display("=== Cache Functional Coverage Report ===");
        $display("Overall Coverage: %.2f%%", cg.get_coverage());
        $display("Operation Coverage: %.2f%%", cg.cp_operation.get_coverage());
        $display("State Coverage: %.2f%%", cg.cp_state.get_coverage());
        $display("Hit/Miss Coverage: %.2f%%", cg.cp_hit_miss.get_coverage());
        $display("Cross Coverage: %.2f%%", cg.cross_state_op.get_coverage());
    endfunction
endclass

// Coverage-driven test generator
class coverage_driven_generator;
    cache_coverage cov;
    mailbox #(cache_transaction) gen2drv;
    int target_coverage = 95;
    int max_iterations = 10000;
    
    function new(cache_coverage cov, mailbox #(cache_transaction) gen2drv);
        this.cov = cov;
        this.gen2drv = gen2drv;
    endfunction
    
    task run();
        cache_transaction trans;
        int iteration = 0;
        
        while (cov.get_coverage() < target_coverage && iteration < max_iterations) begin
            trans = new();
            
            // Bias randomization toward uncovered scenarios
            if (cov.cg.cp_operation.get_coverage() < 90) begin
                // Focus on less covered operations
                trans.constraint_mode(0);
                trans.operation_bias.constraint_mode(1);
            end
            
            if (cov.cg.cross_state_op.get_coverage() < 80) begin
                // Focus on uncovered state/operation combinations
                trans.state_bias.constraint_mode(1);
            end
            
            assert(trans.randomize()) else $fatal("Randomization failed");
            gen2drv.put(trans);
            
            iteration++;
            if (iteration % 100 == 0) begin
                $display("Iteration %0d: Coverage = %.2f%%", iteration, cov.get_coverage());
            end
        end
        
        $display("Coverage-driven generation complete:");
        $display("Final coverage: %.2f%% after %0d iterations", cov.get_coverage(), iteration);
    endtask
endclass
```

#### Assertion-Based Verification

SystemVerilog Assertions (SVA) provide a powerful way to specify and verify design properties directly in the design or testbench.

##### Comprehensive SVA Examples

```systemverilog
module fifo_assertions #(
    parameter DEPTH = 16,
    parameter WIDTH = 32
)(
    input logic clk,
    input logic rst_n,
    input logic wr_en,
    input logic rd_en,
    input logic [WIDTH-1:0] wr_data,
    input logic [WIDTH-1:0] rd_data,
    input logic full,
    input logic empty,
    input logic [$clog2(DEPTH):0] count
);

    // Basic protocol assertions
    
    // Assert: Reset behavior
    property reset_behavior;
        @(posedge clk) !rst_n |-> ##1 (empty && !full && count == 0);
    endproperty
    assert_reset: assert property (reset_behavior)
        else $error("FIFO reset behavior violation at time %0t", $time);
    
    // Assert: Full and empty are mutually exclusive
    property full_empty_exclusive;
        @(posedge clk) !(full && empty);
    endproperty
    assert_full_empty: assert property (full_empty_exclusive)
        else $error("FIFO full and empty both asserted at time %0t", $time);
    
    // Assert: Count consistency with full/empty flags
    property count_consistency;
        @(posedge clk) disable iff (!rst_n)
        (count == 0) <-> empty;
    endproperty
    assert_count_empty: assert property (count_consistency)
        else $error("Count/empty inconsistency: count=%0d, empty=%b at time %0t", 
                   count, empty, $time);
    
    property count_full_consistency;
        @(posedge clk) disable iff (!rst_n)
        (count == DEPTH) <-> full;
    endproperty
    assert_count_full: assert property (count_full_consistency)
        else $error("Count/full inconsistency: count=%0d, full=%b at time %0t", 
                   count, full, $time);
    
    // Assert: No write when full
    property no_write_when_full;
        @(posedge clk) disable iff (!rst_n)
        full |-> !wr_en;
    endproperty
    assert_no_write_full: assert property (no_write_when_full)
        else $error("Write attempted when FIFO full at time %0t", $time);
    
    // Assert: No read when empty
    property no_read_when_empty;
        @(posedge clk) disable iff (!rst_n)
        empty |-> !rd_en;
    endproperty
    assert_no_read_empty: assert property (no_read_when_empty)
        else $error("Read attempted when FIFO empty at time %0t", $time);
    
    // Assert: Count increment on write (when not full)
    property count_increment_write;
        @(posedge clk) disable iff (!rst_n)
        (wr_en && !full && !rd_en) |=> (count == $past(count) + 1);
    endproperty
    assert_count_inc: assert property (count_increment_write)
        else $error("Count did not increment on write at time %0t", $time);
    
    // Assert: Count decrement on read (when not empty)
    property count_decrement_read;
        @(posedge clk) disable iff (!rst_n)
        (rd_en && !empty && !wr_en) |=> (count == $past(count) - 1);
    endproperty
    assert_count_dec: assert property (count_decrement_read)
        else $error("Count did not decrement on read at time %0t", $time);
    
    // Assert: Simultaneous read/write keeps count stable
    property count_stable_rd_wr;
        @(posedge clk) disable iff (!rst_n)
        (wr_en && rd_en && !full && !empty) |=> (count == $past(count));
    endproperty
    assert_count_stable: assert property (count_stable_rd_wr)
        else $error("Count not stable during simultaneous read/write at time %0t", $time);
    
    // Data integrity assertion using associative array
    logic [WIDTH-1:0] shadow_fifo [$];
    
    always @(posedge clk) begin
        if (!rst_n) begin
            shadow_fifo.delete();
        end else begin
            // Track writes
            if (wr_en && !full) begin
                shadow_fifo.push_back(wr_data);
            end
            
            // Check reads
            if (rd_en && !empty) begin
                logic [WIDTH-1:0] expected_data;
                expected_data = shadow_fifo.pop_front();
                
                immediate assert (rd_data === expected_data)
                    else $error("Data integrity violation: expected=%08h, got=%08h at time %0t",
                               expected_data, rd_data, $time);
            end
        end
    end
    
    // Liveness properties
    
    // Assert: Eventually empty after reset
    property eventually_empty_after_reset;
        @(posedge clk) $fell(rst_n) |-> ##[1:100] empty;
    endproperty
    assert_eventually_empty: assert property (eventually_empty_after_reset)
        else $error("FIFO did not become empty within 100 cycles of reset");
    
    // Assert: Data will eventually be read if FIFO is not empty
    property data_eventually_read;
        @(posedge clk) disable iff (!rst_n)
        (!empty && !rd_en) |-> ##[1:50] (rd_en || empty);
    endproperty
    assert_data_read: assert property (data_eventually_read)
        else $warning("Data remained unread for extended period");
    
    // Coverage properties
    cover_full: cover property (@(posedge clk) full);
    cover_empty: cover property (@(posedge clk) empty);
    cover_simultaneous_rd_wr: cover property (@(posedge clk) wr_en && rd_en);
    cover_half_full: cover property (@(posedge clk) count == DEPTH/2);
    
    // Sequence-based assertions
    
    sequence write_sequence;
        wr_en && !full;
    endsequence
    
    sequence read_sequence;
        rd_en && !empty;
    endsequence
    
    // Assert: After 3 consecutive writes, count increases by 3
    property three_writes_count;
        @(posedge clk) disable iff (!rst_n)
        (write_sequence ##1 write_sequence ##1 write_sequence) |=>
        (count >= $past(count, 3) + 3);
    endproperty
    assert_three_writes: assert property (three_writes_count);
    
    // Performance assertions
    
    // Assert: Maximum latency for read after write
    property write_read_latency;
        @(posedge clk) disable iff (!rst_n)
        (wr_en && !full && empty) |-> ##[1:5] rd_en;
    endproperty
    assert_wr_rd_latency: assert property (write_read_latency)
        else $warning("Read latency exceeded 5 cycles after write to empty FIFO");
    
    // Formal verification properties
    
    // Assume: Valid clock
    assume_clock: assume property (@(posedge clk) 1);
    
    // Assume: Reset duration
    assume_reset: assume property ($rose(rst_n) |-> $past(!rst_n, 2));
    
    // Assume: No X/Z values on control signals
    assume_no_x_wr_en: assume property (@(posedge clk) !$isunknown(wr_en));
    assume_no_x_rd_en: assume property (@(posedge clk) !$isunknown(rd_en));
    
endmodule

// Bus protocol assertions
interface axi_assertions_if(
    input logic clk,
    input logic rst_n,
    axi4_lite_if.monitor axi
);

    // Write address channel assertions
    property awvalid_stable;
        @(posedge clk) disable iff (!rst_n)
        (axi.awvalid && !axi.awready) |=> axi.awvalid;
    endproperty
    assert_awvalid_stable: assert property (awvalid_stable)
        else $error("AWVALID not stable during handshake");
    
    property aw_signals_stable;
        @(posedge clk) disable iff (!rst_n)
        (axi.awvalid && !axi.awready) |=> 
        ($stable(axi.awaddr) && $stable(axi.awprot));
    endproperty
    assert_aw_stable: assert property (aw_signals_stable)
        else $error("AW channel signals changed during handshake");
    
    // Write data channel assertions
    property wvalid_stable;
        @(posedge clk) disable iff (!rst_n)
        (axi.wvalid && !axi.wready) |=> axi.wvalid;
    endproperty
    assert_wvalid_stable: assert property (wvalid_stable);
    
    property w_signals_stable;
        @(posedge clk) disable iff (!rst_n)
        (axi.wvalid && !axi.wready) |=> 
        ($stable(axi.wdata) && $stable(axi.wstrb));
    endproperty
    assert_w_stable: assert property (w_signals_stable);
    
    // Write response channel assertions
    property bvalid_stable;
        @(posedge clk) disable iff (!rst_n)
        (axi.bvalid && !axi.bready) |=> axi.bvalid;
    endproperty
    assert_bvalid_stable: assert property (bvalid_stable);
    
    // Transaction ordering assertions
    property write_ordering;
        @(posedge clk) disable iff (!rst_n)
        (axi.awvalid && axi.awready) ##1 (axi.wvalid && axi.wready) |->
        ##[1:10] (axi.bvalid && axi.bready);
    endproperty
    assert_write_order: assert property (write_ordering)
        else $error("Write response not received within expected time");
    
    // Deadlock prevention
    property no_deadlock;
        @(posedge clk) disable iff (!rst_n)
        always ##[1:100] (!axi.awvalid || axi.awready) && 
                         (!axi.wvalid || axi.wready) &&
                         (!axi.arvalid || axi.arready);
    endproperty
    assert_no_deadlock: assert property (no_deadlock)
        else $error("Potential deadlock detected - signals stuck");
    
    // Coverage for interesting scenarios
    cover_write_burst: cover property (
        @(posedge clk) (axi.awvalid && axi.awready) ##1 (axi.wvalid && axi.wready)
    );
    
    cover_back_to_back_writes: cover property (
        @(posedge clk) (axi.bvalid && axi.bready) ##1 (axi.awvalid && axi.awready)
    );
    
    cover_simultaneous_read_write: cover property (
        @(posedge clk) (axi.awvalid && axi.arvalid)
    );

endinterface
```

#### Complete Verification Environment Example

Here's a complete verification environment that ties together all the concepts:

```systemverilog
// Top-level verification environment
class complete_verification_env;
    // Environment components
    alu_generator gen;
    alu_driver drv;
    alu_monitor mon;
    alu_scoreboard sb;
    cache_coverage cov;
    axi4_lite_checker protocol_chk;
    
    // Communication
    mailbox #(alu_transaction) gen2drv;
    mailbox #(alu_transaction) mon2sb;
    
    // Virtual interfaces
    virtual alu_if alu_vif;
    virtual axi4_lite_if axi_vif;
    
    // Configuration
    verification_config cfg;
    
    function new(virtual alu_if alu_vif, virtual axi4_lite_if axi_vif);
        this.alu_vif = alu_vif;
        this.axi_vif = axi_vif;
        
        // Create mailboxes
        gen2drv = new();
        mon2sb = new();
        
        // Create components
        gen = new(gen2drv);
        drv = new(alu_vif, gen2drv);
        mon = new(alu_vif, mon2sb);
        sb = new(mon2sb);
        cov = new();
        protocol_chk = new(axi_vif);
        
        // Configuration
        cfg = new();
    endfunction
    
    task run_test();
        $display("=== Starting Verification Environment ===");
        
        fork
            gen.run();
            drv.run();
            mon.run();
            sb.run();
            protocol_chk.run();
            timeout_watchdog();
        join_any
        
        // Wait for completion and report
        #1000ns;
        report_results();
    endtask
    
    task timeout_watchdog();
        #(cfg.timeout_ns * 1ns);
        $display(" TIMEOUT: Test exceeded %0d ns ", cfg.timeout_ns);
        $finish;
    endtask
    
    task report_results();
        $display("\n" + "="*50);
        $display("VERIFICATION RESULTS SUMMARY");
        $display("="*50);
        
        sb.report();
        cov.report();
        protocol_chk.report();
        
        // Overall pass/fail determination
        bit overall_pass = (sb.fail_count == 0) && 
                          (protocol_chk.addr_phase_violations == 0) &&
                          (protocol_chk.data_phase_violations == 0) &&
                          (protocol_chk.handshake_violations == 0) &&
                          (cov.get_coverage() >= cfg.min_coverage);
        
        $display("\n" + "="*30);
        if (overall_pass) begin
            $display(" VERIFICATION PASSED ");
        end else begin
            $display(" VERIFICATION FAILED ");
        end
        $display("="*30);
        
        $finish;
    endtask
endclass

class verification_config;
    int num_transactions = 1000;
    int timeout_ns = 100000;
    real min_coverage = 90.0;
    bit enable_assertions = 1;
    bit enable_coverage = 1;
    
    function void display();
        $display("Verification Configuration:");
        $display("  Transactions: %0d", num_transactions);
        $display("  Timeout: %0d ns", timeout_ns);
        $display("  Min Coverage: %.1f%%", min_coverage);
        $display("  Assertions: %s", enable_assertions ? "ON" : "OFF");
        $display("  Coverage: %s", enable_coverage ? "ON" : "OFF");
    endfunction
endclass

// Top-level testbench module
module complete_testbench();
    logic clk = 0;
    logic rst_n = 0;
    
    always #5ns clk = ~clk;
    
    initial begin
        rst_n = 0;
        #100ns rst_n = 1;
    end
    
    // Interfaces
    alu_if alu_vif(clk, rst_n);
    axi4_lite_if axi_vif(clk, rst_n);
    
    // DUT instances
    alu #(.WIDTH(8)) alu_dut (
        .a(alu_vif.a),
        .b(alu_vif.b),
        .op(alu_vif.op),
        .result(alu_vif.result),
        .zero(alu_vif.zero),
        .overflow(alu_vif.overflow),
        .carry(alu_vif.carry)
    );
    
    // Assertion bindings
    bind alu_dut alu_assertions alu_assert_inst (
        .clk(clk),
        .rst_n(rst_n),
        .a(a),
        .b(b),
        .op(op),
        .result(result),
        .zero(zero),
        .overflow(overflow),
        .carry(carry)
    );
    
    // Verification environment
    complete_verification_env env;
    
    initial begin
        env = new(alu_vif, axi_vif);
        env.cfg.display();
        env.run_test();
    end
    
    // Dump waves for debugging
    initial begin
        $dumpfile("verification.vcd");
        $dumpvars(0, complete_testbench);
    end
endmodule
```

#### Summary

This chapter provides comprehensive examples of SystemVerilog verification techniques, including:

1. **Complete ALU testbench** with transaction-level modeling, driver, monitor, and scoreboard
2. **Memory controller verification** with sophisticated protocol checking and reference modeling
3. **Bus protocol checkers** using SVA for AXI4-Lite compliance verification
4. **Coverage-driven verification** with functional coverage and directed test generation
5. **Assertion-based verification** with comprehensive property specifications
6. **Complete verification environment** that integrates all components

These examples demonstrate industry-standard verification methodologies and can be adapted for various design verification projects. The code shows proper use of SystemVerilog OOP features, interfaces, assertions, and coverage constructs to create robust and maintainable verification environments.